<a href="https://colab.research.google.com/github/TSGreenwood/tsgreenwood.github.io/blob/google_cloud/map_bike_lanes_folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Map bike lanes using Folium**

In [1]:
# Set the current project in Google Cloud SDK to 'taragreenwoodpersonalwebsite'

!gcloud config set project 'taragreenwoodpersonalwebsite'

Updated property [core/project].


In [2]:
# Connect notebook to my Drive account

from google.colab import auth
auth.authenticate_user()

In [3]:
# Navigate to my repo

!git clone https://github.com/TSGreenwood/tsgreenwood.github.io.git

Cloning into 'tsgreenwood.github.io'...
remote: Enumerating objects: 1357, done.
remote: Counting objects: 100% (1357/1357), done.
remote: Compressing objects: 100% (572/572), done.
remote: Total 1357 (delta 830), reused 1253 (delta 759), pack-reused 0
Receiving objects: 100% (1357/1357), 37.46 MiB | 25.76 MiB/s, done.
Resolving deltas: 100% (830/830), done.


In [4]:
%cd tsgreenwood.github.io/

/content/tsgreenwood.github.io


In [5]:
#! ls

 CNAME		  index.html				    menu.html
 css		  js					   'python_foundation_environment .yml'
 embedding.html   king_county_public_transit_options.html   README.md
 empty.html	  kingcountytransitmapping		    resume.html
 footer.html	  LICENSE				    static
 img		  mapbox_gl_js_tutorial.html		    widgets.html


In [5]:
# Switch branches

!git checkout google_cloud

Branch 'google_cloud' set up to track remote branch 'google_cloud' from 'origin'.
Switched to a new branch 'google_cloud'


In [7]:
# If config problems
#!git config user.email "greenwood.ts@gmail.com"
#!git config user.name "Greenwood"

In [5]:
# Pull the latest changes

!git pull origin google_cloud

From https://github.com/TSGreenwood/tsgreenwood.github.io
 * branch            google_cloud -> FETCH_HEAD
hint: You have divergent branches and need to specify how to reconcile them.
hint: You can do so by running one of the following commands sometime before
hint: your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.
fatal: Need to specify how to reconcile divergent branches.


In [7]:
# Import my mapping functions from the mapping_functions.py file located in my repository into Colab notebook

!wget https://raw.githubusercontent.com/TSGreenwood/tsgreenwood.github.io/google_cloud/kingcountytransitmapping/utils/mapping_functions.py

--2024-03-25 02:28:36--  https://raw.githubusercontent.com/TSGreenwood/tsgreenwood.github.io/google_cloud/kingcountytransitmapping/utils/mapping_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2861 (2.8K) [text/plain]
Saving to: ‘mapping_functions.py’

mapping_functions.p 100%[===================>]   2.79K  --.-KB/s    in 0s      

2024-03-25 02:28:36 (27.1 MB/s) - ‘mapping_functions.py’ saved [2861/2861]



In [8]:
import mapping_functions

In [10]:
! git status

On branch google_cloud
Your branch is up to date with 'origin/google_cloud'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	__pycache__/
	mapping_functions.py

nothing added to commit but untracked files present (use "git add" to track)


In [12]:
# Work directly with GeoJSON dataset stored in Google Cloud bucket

import json
from google.cloud import storage

In [13]:
# Access the GeoJSON File from the Bucket

# Initialize a client
storage_client = storage.Client()

# Define your bucket name and file name
bucket_name = 'kingcountypublictransit'
blob_name = 'data/bike_lanes.geojson'

# Create a bucket object and a blob (file) object
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)

In [14]:
# Ddownload the GeoJSON file directly into memory and load it as a JSON object. This approach avoids saving the file locally

# Download the file's content into memory
geojson_as_string = blob.download_as_string()

# Convert the string to a JSON object
geojson_data = json.loads(geojson_as_string)

Now that the dataset is loaded, we'll do some mapping.

In [9]:
# Initialize basemap

m1 = mapping_functions.create_kc_map()

In [10]:
m1